In [1]:
#LNB Messieurs
#LNB Dames
#Première ligue messieurs, gr. a
#Première ligue dames, gr. a
#Groupe b

In [3]:
import requests
import datetime
header= {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36',
         'Host': 'api.volleyball.ch',
         'Connection': 'keep-alive',
         'Access-Control-Request-Method': 'GET', 
         'Origin': 'https://www.volleyball.ch',
          'Access-Control-Request-Headers': 'authorization',
         'Accept': '*/*', 
         'Referer': 'https://www.volleyball.ch/fr/volleyball/game-center/', 
          'Accept-Encoding': 'gzip, deflate, br', 
         'Accept-Language': 'fr-FR,fr;q=0.9,en-US;q=0.8,en;q=0.7,it;q=0.6',
        'Authorization': 'clicsoftGmbhMasterApiKey'
}
def nettoie(x):
    x = x.replace('VBC', '').replace('VB', '').replace('Volleyball','').replace('Volley', '').strip()
    dico = {'TV Grenchen' : 'Granges', 'Solothurn' : 'Soleure'}
    if x in dico.keys():
        x = dico[x]
    return x

def matches(link):
    def computeScore(x):
        resh, resa = 0, 0
        try:
            for i in range(1, len(data[x]["setResults"]) + 1):
                if data[x]["setResults"]["{}".format(i)]["home"] < data[x]["setResults"]["{}".format(i)]["away"]:
                    resa += 1
                else:
                    resh += 1
        except:
            resh,resa = 'x', 'x'
        return (str(resh) + '-' + str(resa))
    
    def cordate(x):
        jour = datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S').strftime('%A')
        mois = datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S').strftime('%B')
        njour = datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S').strftime('%d')
        heure = datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S').strftime('%H')
        minute = datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S').strftime('%M')

        dictiojour = {'Monday' : 'Lundi',
                 'Tuesday':'Mardi',
                 'Wednesday':'Mercredi',
                 'Thursday':'Jeudi',
                 'Friday':'Vendredi',
                 'Saturday':'Samedi',
                 'Sunday':'Dimanche'}
        dictiomois = {'January':'janvier',
                     'February':'février',
                     'March':'mars',
                     'April':'avril',
                      'May':'mai',
                     'June':'juin',
                     'July':'juillet',
                     'August':'août',
                     'September':'septembre',
                     'October':'octobre',
                     'November':'novembre',
                     'December':'décembre'}
        res = dictiojour.get(jour) + ' ' + str(int(njour)) + ' ' + dictiomois.get(mois)+'. '+ heure +'h'+ minute+':'
        return res
    
    response = requests.get(link, headers=header)
    data = response.json()
    toreturn = ''
    avenir = ''
    for team in range(0, len(data)+1):

        try:
            if datetime.datetime.strptime(data[team]["playDate"],'%Y-%m-%d %H:%M:%S') >= datetime.datetime.now() - datetime.timedelta(days=6, hours=12) and datetime.datetime.strptime(data[team]["playDate"],'%Y-%m-%d %H:%M:%S') <= datetime.datetime.now() + datetime.timedelta(hours=12):
                toreturn += (nettoie(data[team]["teams"]["home"]["caption"]) + ' - '
                      + nettoie(data[team]["teams"]["away"]["caption"])
                      +'\t' + computeScore(team)) + '\n'
            elif datetime.datetime.strptime(data[team]["playDate"],'%Y-%m-%d %H:%M:%S') <= datetime.datetime.now() + datetime.timedelta(days=7, hours=12) and datetime.datetime.strptime(data[team]["playDate"],'%Y-%m-%d %H:%M:%S') >= datetime.datetime.now() + datetime.timedelta(hours=12):
                avenir += cordate(str(datetime.datetime.strptime(data[team]["playDate"],'%Y-%m-%d %H:%M:%S'))) + ' ' + nettoie(data[team]["teams"]["home"]["caption"]) + ' - ' + nettoie(data[team]["teams"]["away"]["caption"]) +'. '

        except:
            pass

    return toreturn, avenir

def classement(link):
    response_ranking = requests.get(link, headers=header)
    data_ranking = response_ranking.json()
    toreturn = ''
    for i in range(len(data_ranking)):
        toreturn+=('\t' + str(i+1) +'.'+ '\t' + nettoie(data_ranking[i]['teamCaption'])
              + '\t' + str(data_ranking[i]['games'])
              + '\t' + str(data_ranking[i]['winsClear'])
              + '\t' + str(data_ranking[i]['winsNarrow'])
              + '\t' + str(data_ranking[i]['defeatsClear'])
              + '\t' + str(data_ranking[i]['defeatsNarrow'])
              + '\t' + str(data_ranking[i]['setsWon'])
              + '-' +  str(data_ranking[i]['setsLost'])
              + '\t' + str(data_ranking[i]['points']) + '\n'
              )
    return toreturn

def clacontinu(link):
    response_ranking = requests.get(link, headers=header)
    data_ranking = response_ranking.json()
    
    if data_ranking[0]['points'] == data_ranking[1]['points']:
        toreturn = 'Classement: ' + str(1) +'.'+ ' ' + nettoie(data_ranking[0]['teamCaption']) + ' ' + str(data_ranking[0]['games']) + '-' + str(data_ranking[0]['points']) + ' ' + '(' + str(data_ranking[0]['setsWon']) + '-' +  str(data_ranking[i]['setsLost']) +'). '
    else:
        toreturn = 'Classement: ' + str(1) +'.'+ ' ' + nettoie(data_ranking[0]['teamCaption']) + ' ' + str(data_ranking[0]['games']) + '-' + str(data_ranking[0]['points']) + '. '
    
    for i in range(1, len(data_ranking)-1):
        if data_ranking[i]['points'] == data_ranking[i+1]['points'] or data_ranking[i]['points'] == data_ranking[i-1]['points']:
            toreturn+= str(i+1) +'.'+ ' ' + nettoie(data_ranking[i]['teamCaption']) + ' ' + str(data_ranking[i]['games']) + '-' + str(data_ranking[i]['points']) + ' ' + '(' + str(data_ranking[i]['setsWon']) + '-' +  str(data_ranking[i]['setsLost']) +'). '
        else:
            toreturn+= str(i+1) +'.'+ ' ' + nettoie(data_ranking[i]['teamCaption']) + ' ' + str(data_ranking[i]['games']) + '-' + str(data_ranking[i]['points']) + '. '
    
    if data_ranking[len(data_ranking)-2]['points'] == data_ranking[len(data_ranking)-1]['points']:
        toreturn+= str(len(data_ranking)) +'.'+ ' ' + nettoie(data_ranking[len(data_ranking)-1]['teamCaption']) + ' ' + str(data_ranking[len(data_ranking)-1]['games']) + '-' + str(data_ranking[len(data_ranking)-1]['points']) + ' ' + '(' + str(data_ranking[len(data_ranking)-1]['setsWon']) + '-' +  str(data_ranking[len(data_ranking)-1]['setsLost']) +'). '
    else:
        toreturn+= str(len(data_ranking)) +'.'+ ' ' + nettoie(data_ranking[len(data_ranking)-1]['teamCaption']) + ' ' + str(data_ranking[len(data_ranking)-1]['games']) + '-' + str(data_ranking[len(data_ranking)-1]['points']) + '. '
            
    return toreturn
    


In [2]:
jouer, a_jouer = matches("https://api.volleyball.ch/indoor/games?region=SV&gender=m&leagueId=1974&phaseId=3856&groupId=12602")
print("LNB Messieurs, relégation")
print(jouer)
print(classement("https://api.volleyball.ch/indoor/ranking/12602"))
print(a_jouer)

LNB Messieurs, relégation
Näfels II - Wetzikon	2-3
Papiermühle - Fully	3-2
Oberdiessbach - Colombier  I	1-3

	1.	Colombier  I	2	1	0	0	1	5-4	6
	2.	Pallavolo Kreuzlingen	1	1	0	0	0	3-0	5
	3.	Papiermühle	1	0	1	0	0	3-2	5
	4.	Wetzikon	2	0	2	0	0	6-4	5
	5.	Näfels II	2	0	0	0	2	4-6	5
	6.	Oberdiessbach	2	0	1	1	0	4-5	2
	7.	Fully	2	0	0	1	1	2-6	2

Samedi 16 février. 14h30: Näfels II - Fully. Samedi 16 février. 16h00: Wetzikon - Papiermühle. Dimanche 17 février. 15h00: Pallavolo Kreuzlingen - Oberdiessbach. 


In [50]:
jouer, a_jouer = matches("https://api.volleyball.ch/indoor/games?region=SV&gender=f&leagueId=1973&phaseId=3858&groupId=12598")
print("LNB Dames, relégation")
print(jouer)
print(classement("https://api.volleyball.ch/indoor/ranking/12598"))
print(a_jouer)

LNB Dames, relégation
Visp - Raiffeisen a Obwalden	x-x

1.	Raiffeisen a Obwalden	1	1	0	0	0	3-0	5
2.	Steinhausen	1	0	1	0	0	3-2	5
3.	Wolf Haus Giubiasco	1	1	0	0	0	3-1	4
4.	Luzern	1	1	0	0	0	3-0	3
5.	NUC II	1	0	0	1	0	1-3	3
6.	Visp	1	0	0	0	1	2-3	2
7.	Gerlafingen	1	0	0	1	0	0-3	2
8.	Edelline Köniz	1	0	0	1	0	0-3	0

Dimanche 10 février. 14h30: NUC II - Luzern. Dimanche 10 février. 16h00: Gerlafingen - Wolf Haus Giubiasco. Dimanche 10 février. 16h30: Edelline Köniz - Steinhausen. Samedi 16 février. 13h30: Wolf Haus Giubiasco - Visp. Samedi 16 février. 16h00: Luzern - Edelline Köniz. Samedi 16 février. 16h00: Steinhausen - NUC II. Samedi 16 février. 18h00: Raiffeisen a Obwalden - Gerlafingen. 


In [53]:
jouer, a_jouer = matches("https://api.volleyball.ch/indoor/games?region=SV&gender=m&leagueId=1976&phaseId=3491&groupId=11798")
print("1ere ligue messieurs, groupe A")
print(jouer)
print(clacontinu("https://api.volleyball.ch/indoor/ranking/11798"))
print(a_jouer)

1ere ligue messieurs, groupe A
TV Murten - Martigny	3-1
Groupement Sportif du CERN - Colombier  II	2-3
La-Chaux-de-Fonds - Herren Oberwallis	x-x
Martigny - La Côte	x-x
Volero Aarberg - Lausanne	x-x

1. Volero Aarberg 16-42. 2. La Côte 16-34. 3. Herren Oberwallis 15-32 (34-19). 4. Servette Star-Onex 16-32 (39-23). 5. Colombier  II 17-27. 6. Lausanne 16-24. 7. La-Chaux-de-Fonds 16-22. 8. TV Murten 16-18. 10. Groupement Sportif du CERN 16-4. 
Dimanche 10 février. 14h00: TV Murten - Servette Star-Onex. Dimanche 10 février. 16h00: Herren Oberwallis - Groupement Sportif du CERN. Vendredi 15 février. 20h30: Servette Star-Onex - Groupement Sportif du CERN. Vendredi 15 février. 20h30: Lausanne - TV Murten. Samedi 16 février. 15h30: Colombier  II - Martigny. Samedi 16 février. 17h30: La Côte - La-Chaux-de-Fonds. Samedi 16 février. 18h00: Herren Oberwallis - Volero Aarberg. 


In [54]:
jouer, a_jouer = matches("https://api.volleyball.ch/indoor/games?region=SV&gender=f&leagueId=1975&phaseId=3486&groupId=11785")
print("1ere ligue dames, groupe A")
print(jouer)
print(clacontinu("https://api.volleyball.ch/indoor/ranking/11785"))
print(a_jouer)

1ere ligue dames, groupe A
TV Murten - Cheseaux II	3-0
LE Locle - VC Uettligen	0-3
Cheseaux II - Ecublens	x-x
TV Murten - Servette Star-Onex	x-x

1. Val-de-Ruz Sport 15-38. 2. Ecublens 14-36. 3. TS  Düdingen II 14-31. 4. Servette Star-Onex 14-22. 5. TV Murten 14-19. 6. Cheseaux II 14-18. 7. Genève  II 15-15 (21-34). 9. LE Locle 15-1. 
Dimanche 10 février. 13h30: TS  Düdingen II - Val-de-Ruz Sport. Vendredi 15 février. 20h30: Ecublens - TS  Düdingen II. Samedi 16 février. 12h00: Servette Star-Onex - LE Locle. Samedi 16 février. 14h00: Genève  II - TV Murten. Samedi 16 février. 18h30: VC Uettligen - Cheseaux II. 


In [4]:
jouer, a_jouer = matches("https://api.volleyball.ch/indoor/games?region=SV&gender=f&leagueId=1975&phaseId=3486&groupId=11786")
print("Groupe B")
print(jouer)
print(clacontinu("https://api.volleyball.ch/indoor/ranking/11786"))
print(a_jouer)

Groupe B
Uni Bern - VFM -  Franches-Montagnes II	2-3
Münchenbuchsee - Soleure	3-0
Lalden - Oftringen 1	3-2
Köniz II - Muri Bern	3-1
Münsingen - Granges	3-2

Classement: 1. Münsingen 17-40. 2. Muri Bern 17-39. 3. VFM -  Franches-Montagnes II 17-35. 4. Münchenbuchsee 17-34. 5. Granges 17-32. 6. Köniz II 17-30. 7. Uni Bern 17-25. 8. Lalden 17-10. 9. Oftringen 1 17-7. 10. Soleure 17-3. 
Samedi 16 février. 13h30: Muri Bern - Münchenbuchsee. Samedi 16 février. 13h45: Granges - Köniz II. Samedi 16 février. 14h30: Soleure - Lalden. Samedi 16 février. 15h00: VFM -  Franches-Montagnes II - Münsingen. Samedi 16 février. 18h00: Oftringen 1 - Uni Bern. 
